# Company1 - ETL Process

This notebook is organized in the following sections:
* [Step 0 - Preliminary: Viewing the data](#0)
* [Step 1 - Checking for duplicates](#1)
* [Step 2 - Checking for missing values](#2)
* [Step 3 - Imputing/dropping missing values](#3)
* [Step 4 - Ensuring Correct Datatypes](#4)
* [Step 5 - Preparation for merging](#5)


<a id='0'></a>
## Step 0 - Preliminary: Viewing the data

In [17]:
import pandas as pd

In [18]:
# Load the share prices csv

prices = pd.read_csv("data/us-shareprices-daily.csv",delimiter=';')

In [19]:
prices.tail()

,Ticker,SimFinId,Date,Open,High,Low,Close,Adj. Close,Volume,Dividend,Shares Outstanding
5767354,ZYXI,171401,2024-02-26,13.04,13.04,12.67,12.82,12.82,335055,NaN,36435000.0
5767355,ZYXI,171401,2024-02-27,12.83,13.77,12.83,13.74,13.74,395525,NaN,36435000.0
5767356,ZYXI,171401,2024-02-28,13.63,13.70,13.38,13.49,13.49,290887,NaN,32170182.0
5767357,ZYXI,171401,2024-02-29,13.51,13.57,13.28,13.56,13.56,232534,NaN,32170182.0
5767358,ZYXI,171401,2024-03-01,12.05,13.43,12.00,12.30,12.30,1216112,NaN,32170182.0


In [20]:
# Load the companies csv 
companies = pd.read_csv("data/us-companies.csv",delimiter=';')

Selecting a company:

In [55]:
company_bruker = companies[companies["Company Name"] == "MICROSOFT CORP"]
company_bruker = company_bruker.copy()
company_bruker

,Ticker,SimFinId,Company Name,IndustryId,ISIN,End of financial year (month),Number Employees,Business Summary,Market,CIK,Main Currency
3650,MSFT,59265,MICROSOFT CORP,101003.0,US5949181045,6.0,166475.0,Microsoft Corp is a technology company. It dev...,us,789019.0,USD


In [56]:
prices_bruker = prices[prices["SimFinId"].isin([1253240, 111052, 63877,56317, 59265])]
prices_bruker = prices_bruker.copy()
prices_bruker

,Ticker,SimFinId,Date,Open,High,Low,Close,Adj. Close,Volume,Dividend,Shares Outstanding
14253,AAPL,111052,2019-04-01,47.91,47.92,47.09,47.81,45.81,111447856,NaN,1.842914e+10
14254,AAPL,111052,2019-04-02,47.77,48.62,47.76,48.51,46.47,91062928,NaN,1.842914e+10
14255,AAPL,111052,2019-04-03,48.31,49.12,48.29,48.84,46.79,93087320,NaN,1.842914e+10
14256,AAPL,111052,2019-04-04,48.70,49.09,48.28,48.92,46.87,76457100,NaN,1.842914e+10
14257,AAPL,111052,2019-04-05,49.11,49.27,48.98,49.25,47.19,74106576,NaN,1.842914e+10
...,...,...,...,...,...,...,...,...,...,...,...
5192433,TSLA,56317,2024-02-26,192.29,201.78,192.00,199.40,199.40,111747116,NaN,3.184790e+09
5192434,TSLA,56317,2024-02-27,204.04,205.60,198.26,199.73,199.73,108645412,NaN,3.184790e+09
5192435,TSLA,56317,2024-02-28,200.42,205.30,198.44,202.04,202.04,99806173,NaN,3.184790e+09
5192436,TSLA,56317,2024-02-29,204.18,205.28,198.45,201.88,201.88,85906974,NaN,3.184790e+09


<a id='1'></a>
## Step 1 - Checking for duplicates

In [57]:
#Check for share prices

prices.duplicated().any()

False

In [24]:
# Check for companies 

companies.duplicated().any()

False

Conclusion: There are no duplicate values.

<a id='2'></a>
## Step 2 - Checking for missing values

**Checking in specific company:**

In [58]:
prices_bruker.isna().sum() / len(prices_bruker)

Ticker                0.000000
SimFinId              0.000000
Date                  0.000000
Open                  0.000000
High                  0.000000
Low                   0.000000
Close                 0.000000
Adj. Close            0.000000
Volume                0.000000
Dividend              0.987086
Shares Outstanding    0.000000
dtype: float64

In [28]:
company_bruker.isna().sum() / len(company_bruker)

Ticker                           0.0
SimFinId                         0.0
Company Name                     0.0
IndustryId                       0.0
ISIN                             0.0
End of financial year (month)    0.0
Number Employees                 0.0
Business Summary                 0.0
Market                           0.0
CIK                              0.0
Main Currency                    0.0
dtype: float64

Conclusion: The only missing values are dividends in the prices_bruker df, which are not paid out all the time. We can assume they equal 0 when they are missing.

<a id='3'></a>
## Step 3 - Imputing/dropping missing values

In [29]:
prices_bruker["Dividend"] = prices_bruker["Dividend"].fillna(0)

In [30]:
prices_bruker.Dividend.unique()

array([0.  , 0.19, 0.2 , 0.22, 0.23, 0.24, 0.32, 0.36, 0.45, 0.47, 0.51,
       0.55, 0.04, 0.05])

In [31]:
prices_bruker.isna().sum() / len(prices_bruker)

Ticker                0.000000
SimFinId              0.000000
Date                  0.000000
Open                  0.000000
High                  0.000000
Low                   0.000000
Close                 0.000000
Adj. Close            0.000000
Volume                0.000000
Dividend              0.000000
Shares Outstanding    0.047458
dtype: float64

Conclusion: No missing values anymore.

<a id='4'></a>
## Step 4 - Ensuring correct datatypes

In [32]:
prices_bruker.info()


<class 'pandas.core.frame.DataFrame'>
Index: 6195 entries, 14253 to 5192437
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Ticker              6195 non-null   object 
 1   SimFinId            6195 non-null   int64  
 2   Date                6195 non-null   object 
 3   Open                6195 non-null   float64
 4   High                6195 non-null   float64
 5   Low                 6195 non-null   float64
 6   Close               6195 non-null   float64
 7   Adj. Close          6195 non-null   float64
 8   Volume              6195 non-null   int64  
 9   Dividend            6195 non-null   float64
 10  Shares Outstanding  5901 non-null   float64
dtypes: float64(7), int64(2), object(2)
memory usage: 580.8+ KB


In [33]:
prices_bruker

,Ticker,SimFinId,Date,Open,High,Low,Close,Adj. Close,Volume,Dividend,Shares Outstanding
14253,AAPL,111052,2019-04-01,47.91,47.92,47.09,47.81,45.81,111447856,0.0,1.842914e+10
14254,AAPL,111052,2019-04-02,47.77,48.62,47.76,48.51,46.47,91062928,0.0,1.842914e+10
14255,AAPL,111052,2019-04-03,48.31,49.12,48.29,48.84,46.79,93087320,0.0,1.842914e+10
14256,AAPL,111052,2019-04-04,48.70,49.09,48.28,48.92,46.87,76457100,0.0,1.842914e+10
14257,AAPL,111052,2019-04-05,49.11,49.27,48.98,49.25,47.19,74106576,0.0,1.842914e+10
...,...,...,...,...,...,...,...,...,...,...,...
5192433,TSLA,56317,2024-02-26,192.29,201.78,192.00,199.40,199.40,111747116,0.0,3.184790e+09
5192434,TSLA,56317,2024-02-27,204.04,205.60,198.26,199.73,199.73,108645412,0.0,3.184790e+09
5192435,TSLA,56317,2024-02-28,200.42,205.30,198.44,202.04,202.04,99806173,0.0,3.184790e+09
5192436,TSLA,56317,2024-02-29,204.18,205.28,198.45,201.88,201.88,85906974,0.0,3.184790e+09


We see there are is a float64 which might be cleaner to format as int64 (Shares Outstanding) Let's double check, we'll do this for the whole dataset.

In [34]:
prices[prices["Shares Outstanding"] % 1 != 0]["Shares Outstanding"].unique()

array([nan])

Repeating the same checks for company_bruker:

In [35]:
company_bruker.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 1817 to 1817
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Ticker                         1 non-null      object 
 1   SimFinId                       1 non-null      int64  
 2   Company Name                   1 non-null      object 
 3   IndustryId                     1 non-null      float64
 4   ISIN                           1 non-null      object 
 5   End of financial year (month)  1 non-null      float64
 6   Number Employees               1 non-null      float64
 7   Business Summary               1 non-null      object 
 8   Market                         1 non-null      object 
 9   CIK                            1 non-null      float64
 10  Main Currency                  1 non-null      object 
dtypes: float64(4), int64(1), object(6)
memory usage: 96.0+ bytes


In [36]:
company_bruker.head()

,Ticker,SimFinId,Company Name,IndustryId,ISIN,End of financial year (month),Number Employees,Business Summary,Market,CIK,Main Currency
1817,EDU,17708500,New Oriental Education & Technology Group Inc.,102006.0,US6475811070,12.0,46653.0,New Oriental Education & Technology Group Inc....,us,1372920.0,USD


We see there are some float64 which might be cleaner to format as int64, column headers such as ID's, which can only be full numbers. Let's double check, we'll do this for the whole dataset.

In [37]:
companies[companies["IndustryId"] % 1 != 0]["IndustryId"].unique()

array([nan])

In [38]:
companies[companies["End of financial year (month)"] % 1 != 0]["End of financial year (month)"].unique()

array([nan])

In [39]:
companies[companies["Number Employees"] % 1 != 0]["Number Employees"].unique()

array([nan])

In [40]:
companies[companies["CIK"] % 1 != 0]["CIK"].unique()

array([nan])

Conclusion:
- prices_bruker: date column is object and not date format, will be changed + "Shares Outstanding" can be changed to int
- company_bruker: we can convert all floats to integer, to have a cleaner dataset:
    - IndustryId
    - End of financial year (month)
    - Number Employees
    - CIK


Comment:
There are missing values in the whole dataset. If more companies are added to the model, these need to be handled.

In [41]:
# for company_bruker

company_bruker["IndustryId"] =  company_bruker["IndustryId"].astype(int)
company_bruker["End of financial year (month)"] =  company_bruker["End of financial year (month)"].astype(int)
company_bruker["Number Employees"] =  company_bruker["Number Employees"].astype(int)
company_bruker["CIK"] =  company_bruker["CIK"].astype(int)


In [42]:
# for prices_bruker

prices_bruker["Shares Outstanding"] =  prices_bruker["Shares Outstanding"].astype(int)
prices_bruker["Date"] = pd.to_datetime(prices_bruker.Date, format="%Y-%m-%d")

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

<a id='5'></a>
## Step 5 - Load csv for ML preparation

In [64]:
prices_bruker.head()

,Ticker,SimFinId,Date,Open,High,Low,Close,Adj. Close,Volume,Dividend,Shares Outstanding
812641,BRKR,1253240,2019-04-01,38.76,39.01,38.47,38.87,38.20,761845,0.0,156814676
812642,BRKR,1253240,2019-04-02,38.94,39.01,38.39,38.53,37.86,666801,0.0,156814676
812643,BRKR,1253240,2019-04-03,38.74,39.24,38.63,39.01,38.33,2647355,0.0,156814676
812644,BRKR,1253240,2019-04-04,38.95,39.01,37.82,38.32,37.66,503730,0.0,156814676
812645,BRKR,1253240,2019-04-05,38.41,39.23,38.30,38.41,37.74,509516,0.0,156814676


In [65]:
prices_bruker.to_csv("pricesbruker_output.csv")